In [1]:
import open3d as o3d
import numpy as np
import random
import pandas as pd
import math
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named '_tkinter'

# build_2D_map

In [21]:
%matplotlib qt
points = np.load("../semantic_3d_pointcloud/point.npy")
colors = np.load("../semantic_3d_pointcloud/color01.npy")
points = points * 10000. / 255
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points)
point_cloud.colors = o3d.utility.Vector3dVector(colors)
print(point_cloud.get_axis_aligned_bounding_box())
floor = o3d.geometry.AxisAlignedBoundingBox(np.array([-50, -1.3, -50]).T, np.array([50, -0.1, 50]).T)
point_cloud = point_cloud.crop(floor)
points = np.array(point_cloud.points)
colors = np.array(point_cloud.colors)
np.where(colors == (255,0,0))#color red's index
#x[index], z[index]
x = points[:,0]#y
z = points[:,2]#x
plt.scatter(z,x, c=colors)
plt.savefig("../Map/map.png")
plt.show()
start = plt.ginput(1)[0]

AxisAlignedBoundingBox: min: (-3.09137, -1.77533, -4.94901), max: (6.24393, 2.36511, 9.91245)


INFO - 2021-11-26 21:04:05,225 - blocking_input - Event 1
INFO - 2021-11-26 21:04:05,226 - blocking_input - input 1: 5.154035, 1.156099


# RRT

In [22]:
class RRT:
    def __init__(self, points, colors, startPoint, goal, goalRadius = 0.5, stepDistance = 0.5):
        # self.image = cv2.imread(imagePath)
        # self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        self.points = points#(z,x)
        self.colors = colors
        self.startPoint = startPoint
        self.goal = goal
        self.goalRadius = goalRadius
        self.stepDistance = stepDistance
        self.graph = {
            startPoint : [startPoint]
        }

    def getRandomPoint(self):
        x = random.uniform(-5, 10)
        y = random.uniform(-4, 6)
        return (x, y)

    def getNearest(self, point):
        nodes = list(self.graph.keys())
        nodes = np.asarray(nodes)
        point = np.array(point)
        nearest = np.sum((nodes - point)**2, axis=1)
        nearest = np.argmin(nearest)
        return tuple(nodes[nearest])
    
    def isCollision(self, newNode_list):
        # for point in newNode_list:
        #     if(list(self.image[point[1]][point[0]]) != [255,255,255] and list(self.image[point[1]][point[0]]) != [255,0,0] and list(self.image[point[1]][point[0]]) != [0,0,0]):
        #         # print(f"({point[0]}, {point[1]}) = {self.image[point[1]][point[0]]}")
        #         return True
        # return False

        for point in newNode_list:
            point = np.array(point).reshape(1,2)
            
            distance = self.points - point
            distance = (distance[:,0]**2 + distance[:,1]**2)**0.5
            # print(distance)
            collosion = np.where(distance < 0.1, True, False)
            if(True in collosion):
                return True
        return False

    def step(self, randomPoint, nearest):
        
        (dx, dy) = (randomPoint[0]-nearest[0], randomPoint[1] - nearest[1])
        theta = math.atan2(dy, dx)
        newNode_list = list()
        for i in np.arange(0.1, self.stepDistance, 0.1):
            (x,y) = (nearest[0] + i * math.cos(theta), nearest[1] + i * math.sin(theta))
            newNode_list.append((x,y))
        
        return newNode_list

    def addPoint(self, parent, point): 
        if(point not in self.graph[parent]):#don't add same point
            self.graph[parent].append(point)#add point into parent
            self.graph[point] = []#point now is a parent can be search with getNearest

    def isArrivGoal(self, point):
        distance = ( (self.goal[0] - point[0])**2 + (self.goal[1] - point[1])**2 ) **0.5
        if(distance <= self.goalRadius):
            return True
        else:
            return False

    def getPath(self, point):
        path = [point]
        # print(self.graph)
        parenList = self.graph.keys()
        while(point != self.startPoint):
            for  parent in parenList:
                childs = self.graph[parent]
                if(point in childs):
                    path.append(parent)
                    point = parent
                    break

        return path

    def pathTo_txt(self, path):
        file = open("../Map/path.txt", "w")
        for i in path:
            file.write(f"{i[0]},{i[1]}\n")
        file.close()
    
    def start(self):
        index = 0
        plt.scatter(self.points[:,0], self.points[:,1], s=1, c = self.colors)
        plt.scatter(x=self.startPoint[0], y=self.startPoint[1], s =5, c="r")
        plt.scatter(x=self.goal[0], y=self.goal[1], s =5, c="g")

        while(True):
            randomPoint = self.getRandomPoint()
            nearest = self.getNearest(randomPoint)
            newPoint_list = self.step(randomPoint, nearest)#take a step
            newPoint = newPoint_list[-1]
            
            if(not self.isCollision(newPoint_list)):#if there are no obstacle collision
                self.addPoint(nearest, newPoint)
                plt.scatter(x=newPoint[0], y=newPoint[1], s =5, c="r")
                plt.plot([nearest[0],newPoint[0]], [nearest[1], newPoint[1]] , c="black")
                if(self.isArrivGoal(newPoint)):
                    print("GOAL")
                    break
            
            index += 1
        
        #TODO
        #find the path
        path = self.getPath(newPoint)
        # print(path)
        path = path[::-1]#reverse the path
        self.pathTo_txt(path)
        # print(path)
        for i in range(len(path) - 1):
            plt.plot([path[i][0], path[i+1][0]], [path[i][1], path[i+1][1]], c='g')
        print("Graph ploted")
        plt.savefig("../Map/route.png")
        plt.show()
        
        return path
        # plt.imsave("../Map/route.png", self.image)

In [31]:
def getCentroid(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    return int(sum_x/length), int(sum_y/length)

def getGoal(points, colors, target:str):
    df = pd.read_csv("./color_coding_semantic_segmentation_classes.xlsx - Sheet1.csv")
    targetList = {row["Name"]:row["Color_Code (R,G,B)"].replace("(", "").replace(")", "") for _, row in df.iterrows()}
    color = targetList[target]
    color = tuple(map(int, color.split(",")))
    colors = colors*255
    
    targetPoints = np.where(np.all(colors == color, axis=1))
    x = points[:,0][targetPoints[0]]
    y = points[:,1][targetPoints[0]]
    plt.scatter(x,y,c='r')
    
    length = x.shape[0]
    sum_x = np.sum(x)
    sum_y = np.sum(y)
    center = (sum_x/length, sum_y/length)
    
    plt.scatter(center[0], center[1], c="black")
    plt.xlim(-3.09137, 6.24393)
    plt.ylim(-4.94901, 9.9124)
    plt.show()
    return center

point = np.column_stack([z,x])
goal = getGoal(point, colors, "refrigerator")

In [35]:
point = np.column_stack([z,x])
rrt = RRT(point, colors, start, goal, goalRadius=0.8, stepDistance=1.0)
path = rrt.start()


GOAL
Graph ploted


# Navigation

In [36]:
import load
load.start_navigate(path, "refrigerator")

=================Start Navigation=================
================= 1 / 14 =================
================= 2 / 14 =================
================= 3 / 14 =================
================= 4 / 14 =================
================= 5 / 14 =================
================= 6 / 14 =================
================= 7 / 14 =================
================= 8 / 14 =================
================= 9 / 14 =================
================= 10 / 14 =================
================= 11 / 14 =================
================= 12 / 14 =================
================= 13 / 14 =================
=================Start Making Video=================
######## 1 / 1746 ########
######## 2 / 1746 ########
######## 3 / 1746 ########
######## 4 / 1746 ########
######## 5 / 1746 ########
######## 6 / 1746 ########
######## 7 / 1746 ########
######## 8 / 1746 ########
######## 9 / 1746 ########
######## 10 / 1746 ########
######## 11 / 1746 ########
######## 12 / 1746 ########
######

In [1]:
! pip list

Package                 Version
----------------------- ------------------
absl-py                 0.14.1
addict                  2.4.0
appnope                 0.1.2
attrs                   21.2.0
backcall                0.2.0
braceexpand             0.1.7
cachetools              4.2.4
certifi                 2021.10.8
charset-normalizer      2.0.7
click                   8.0.3
cloudpickle             2.0.0
cycler                  0.10.0
debugpy                 1.5.0
decorator               4.4.2
entrypoints             0.3
gitdb                   4.0.7
GitPython               3.1.24
google-auth             1.35.0
google-auth-oauthlib    0.4.6
grpcio                  1.41.0
gym                     0.21.0
habitat-sim             0.2.1
idna                    3.3
ifcfg                   0.22
imageio                 2.9.0
imageio-ffmpeg          0.4.5
ipykernel               6.4.1
ipython                 7.28.0
ipython-genutils        0.2.0
jedi                    0.18.0
joblib           